# Advanced Data Science - Capstone Project

## Feature Engineering - V.1.0

### Carlos Granados

For this project I decided to use solar power generation data uploaded by [Ani Kannal](https://www.kaggle.com/anikannal "Ani Kannal Profile") in [kaggle.com website](https://www.kaggle.com/anikannal/solar-power-generation-data?select=Plant_1_Generation_Data.csv "Data Set from kaggle"). The idea is to predict the power generation given different weather conditions, as temperature and irradiation, and check the health of the devices.

#### NOTE:
Since the data sets are almost ready to be used, the ETL was done on the data exploration step (with the date features), and the data cleansing was not necessary.

A small part of the feature engineering was already performed, joining the data frames for individual sensors...

### 1. Load Libraries:

In [1]:
# Standard python libraries
import numpy as np
import types
import datetime as dt
from scipy import stats

# pandas
import pandas as pd

# Libraries to make plots and related
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.dates import DateFormatter

%matplotlib inline

# Others, required by IBM Watson
import ibm_boto3
from botocore.client import Config

### 2. Re-Read and Check Data Frames

Before to start, lets repeat a couple of steps performed in the *Data Exploration* step.

Some cells are hidden, because they contain sensitive information, as some keys and passwords. The original data files, in CSV format, are already uploaded to IBM Watson project, and they are called from the notebook directly.

In [2]:
# @hidden_cell
#Created by IBM Watson, to load the data files already present in the project assets

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.


In [3]:
# @hidden_cell

# Plant 1, generation and sensor data


# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body0, "__iter__"): body0.__iter__ = types.MethodType( __iter__, body0 )
    
 add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body1, "__iter__"): body1.__iter__ = types.MethodType( __iter__, body1 )

In [4]:
# Read generation data for Plant 1
df_plant1_gen = pd.read_csv(body0)

# Read sensor data for Plant 1
df_plant1_sen = pd.read_csv(body1)

print(df_plant1_gen.shape)
print(df_plant1_sen.shape)

(68778, 7)
(3182, 6)


In [5]:
# @hidden_cell

# Plant 2, generation and sensor data

#add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body0, "__iter__"): body0.__iter__ = types.MethodType( __iter__, body0 )

# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body1, "__iter__"): body1.__iter__ = types.MethodType( __iter__, body1 )

In [6]:
# Read generation data for Plant 2
df_plant2_gen = pd.read_csv(body0)

# Read sensor data for Plant 2
df_plant2_sen = pd.read_csv(body1)

print(df_plant2_gen.shape)
print(df_plant2_sen.shape)

(67698, 7)
(3259, 6)


### 2.1 ETL

As part pf the ETL step, the Date features must be transformed to the correct format

In [7]:
df_plant1_gen['DATE_TIME'] = pd.to_datetime(df_plant1_gen['DATE_TIME'])
df_plant1_sen['DATE_TIME'] = pd.to_datetime(df_plant1_sen['DATE_TIME'])
df_plant2_gen['DATE_TIME'] = pd.to_datetime(df_plant2_gen['DATE_TIME'])
df_plant2_sen['DATE_TIME'] = pd.to_datetime(df_plant2_sen['DATE_TIME'])

### 2.2 Data Cleansing

Now, we see that the DF contains a column called "SOURCE_KEYS", which stands for the inverter ID, basically where the measurement were made. There are a set of them...

In [8]:
df_sub = df_plant1_gen['SOURCE_KEY'].value_counts().reset_index()

There are several values for "SOURCE_KEY"! (22 in total!). So the DFs are split and only three sources are considered, for the sake of simplicity...

Plant 1:

In [9]:
df_plant1a_gen = df_plant1_gen[df_plant1_gen['SOURCE_KEY'] == df_sub['index'][0]]
df_plant1b_gen = df_plant1_gen[df_plant1_gen['SOURCE_KEY'] == df_sub['index'][1]]
df_plant1c_gen = df_plant1_gen[df_plant1_gen['SOURCE_KEY'] == df_sub['index'][2]]

Plant 2:

In [10]:
df_sub = df_plant2_gen['SOURCE_KEY'].value_counts().reset_index()

In [11]:
df_plant2a_gen = df_plant2_gen[df_plant2_gen['SOURCE_KEY'] == df_sub['index'][0]]
df_plant2b_gen = df_plant2_gen[df_plant2_gen['SOURCE_KEY'] == df_sub['index'][1]]
df_plant2c_gen = df_plant2_gen[df_plant2_gen['SOURCE_KEY'] == df_sub['index'][2]]

A final problem is that the shape of both generation data and sensor data is not the same. We need to join both data sets, considering data measured at the same time. We can use the `merge` function

Plant 1:

In [12]:
df_plant1a = df_plant1a_gen.merge(df_plant1_sen, on='DATE_TIME', how='left')
df_plant1b = df_plant1b_gen.merge(df_plant1_sen, on='DATE_TIME', how='left')
df_plant1c = df_plant1c_gen.merge(df_plant1_sen, on='DATE_TIME', how='left')

In [13]:
print(df_plant1a.shape)
print(df_plant1b.shape)
print(df_plant1c.shape)

(3155, 12)
(3154, 12)
(3133, 12)


Plant 2:

In [14]:
df_plant2a = df_plant2a_gen.merge(df_plant2_sen, on='DATE_TIME', how='left')
df_plant2b = df_plant2b_gen.merge(df_plant2_sen, on='DATE_TIME', how='left')
df_plant2c = df_plant2c_gen.merge(df_plant2_sen, on='DATE_TIME', how='left')

In [15]:
print(df_plant2a.shape)
print(df_plant2b.shape)
print(df_plant2c.shape)

(3259, 12)
(3259, 12)
(3259, 12)


### 3. Feature Engineering

Now we start looking at the features on the DFs. To do so, we use the `info` function, in order to see the name of each feature and its data type.

In [16]:
df_plant1a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3155 entries, 0 to 3154
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   DATE_TIME            3155 non-null   datetime64[ns]
 1   PLANT_ID_x           3155 non-null   int64         
 2   SOURCE_KEY_x         3155 non-null   object        
 3   DC_POWER             3155 non-null   float64       
 4   AC_POWER             3155 non-null   float64       
 5   DAILY_YIELD          3155 non-null   float64       
 6   TOTAL_YIELD          3155 non-null   float64       
 7   PLANT_ID_y           3155 non-null   int64         
 8   SOURCE_KEY_y         3155 non-null   object        
 9   AMBIENT_TEMPERATURE  3155 non-null   float64       
 10  MODULE_TEMPERATURE   3155 non-null   float64       
 11  IRRADIATION          3155 non-null   float64       
dtypes: datetime64[ns](1), float64(7), int64(2), object(2)
memory usage: 320.4+ KB


#### 3.1 Date, Month and Time...

The Data time contains several values, as year, month, day, time...

In [17]:
df_plant1a['DATE_TIME'][0]

Timestamp('2020-05-15 00:00:00')

So, let's create two new features, one for the month and one for the hour of the day, both have an influence on the power generated by the solar panel, particularly the time!

To do we use the function `DatetimeIndex` of `pandas`, specifying what do we need, if the month of the time...

In [18]:
df_plant1a['MONTH'] = pd.DatetimeIndex(df_plant1a['DATE_TIME']).month
df_plant1a['DT_TIME'] = pd.to_datetime(df_plant1a['DATE_TIME'])

Let's see what do we have now

In [19]:
df_plant1a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3155 entries, 0 to 3154
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   DATE_TIME            3155 non-null   datetime64[ns]
 1   PLANT_ID_x           3155 non-null   int64         
 2   SOURCE_KEY_x         3155 non-null   object        
 3   DC_POWER             3155 non-null   float64       
 4   AC_POWER             3155 non-null   float64       
 5   DAILY_YIELD          3155 non-null   float64       
 6   TOTAL_YIELD          3155 non-null   float64       
 7   PLANT_ID_y           3155 non-null   int64         
 8   SOURCE_KEY_y         3155 non-null   object        
 9   AMBIENT_TEMPERATURE  3155 non-null   float64       
 10  MODULE_TEMPERATURE   3155 non-null   float64       
 11  IRRADIATION          3155 non-null   float64       
 12  MONTH                3155 non-null   int64         
 13  DT_TIME              3155 non-nul

But how looks like the new DT_TIME feature?

In [20]:
df_plant1a['DT_TIME'][0]

Timestamp('2020-05-15 00:00:00')

It's a datetime value, as a Timestamp object!

Let's transform it to a Datetime value and then to a float number, indicating the "absolute" hour of the day (float between 0.0 and 24.0)

In [21]:
dfa = df_plant1a['DT_TIME']
df_plant1a['TIME'] = dfa.dt.hour + dfa.dt.minute/60.0

Now we have an absolute time!

We do now the same for the other DFs!

In [22]:
# We add the month

# Plant 1 B-C:
df_plant1b['MONTH'] = pd.DatetimeIndex(df_plant1b['DATE_TIME']).month
df_plant1c['MONTH'] = pd.DatetimeIndex(df_plant1c['DATE_TIME']).month

# Plant 2 A-C:
df_plant2a['MONTH'] = pd.DatetimeIndex(df_plant2a['DATE_TIME']).month
df_plant2b['MONTH'] = pd.DatetimeIndex(df_plant2b['DATE_TIME']).month
df_plant2c['MONTH'] = pd.DatetimeIndex(df_plant2c['DATE_TIME']).month

# We add the day
# Plant 1 A-C:
df_plant1a['DAY'] = pd.DatetimeIndex(df_plant1a['DATE_TIME']).day
df_plant1b['DAY'] = pd.DatetimeIndex(df_plant1b['DATE_TIME']).day
df_plant1c['DAY'] = pd.DatetimeIndex(df_plant1c['DATE_TIME']).day

# Plant 2 A-C:
df_plant2a['DAY'] = pd.DatetimeIndex(df_plant2a['DATE_TIME']).day
df_plant2b['DAY'] = pd.DatetimeIndex(df_plant2b['DATE_TIME']).day
df_plant2c['DAY'] = pd.DatetimeIndex(df_plant2c['DATE_TIME']).day

# We add the Additional DT_TIME:
# Plant 1 B-C:
df_plant1b['DT_TIME'] = pd.to_datetime(df_plant1b['DATE_TIME'])
df_plant1c['DT_TIME'] = pd.to_datetime(df_plant1c['DATE_TIME'])

# Plant 2 A-C:
df_plant2a['DT_TIME'] = pd.to_datetime(df_plant2a['DATE_TIME'])
df_plant2b['DT_TIME'] = pd.to_datetime(df_plant2b['DATE_TIME'])
df_plant2c['DT_TIME'] = pd.to_datetime(df_plant2c['DATE_TIME'])

And finally the "absolute" time...

In [23]:
# Plant 1 B-C:
dfb, dfc = df_plant1b['DT_TIME'], df_plant1c['DT_TIME']
df_plant1b['TIME'] = dfb.dt.hour + dfb.dt.minute/60.0
df_plant1c['TIME'] = dfc.dt.hour + dfc.dt.minute/60.0

# Plant 2 A-C:
dfa, dfb, dfc = df_plant2a['DT_TIME'], df_plant2b['DT_TIME'], df_plant2c['DT_TIME']
df_plant2a['TIME'] = dfa.dt.hour + dfa.dt.minute/60.0
df_plant2b['TIME'] = dfb.dt.hour + dfb.dt.minute/60.0
df_plant2c['TIME'] = dfc.dt.hour + dfc.dt.minute/60.0

Let's see what do we have...

In [24]:
df_plant1a.head()

,DATE_TIME,PLANT_ID_x,SOURCE_KEY_x,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,PLANT_ID_y,SOURCE_KEY_y,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,MONTH,DT_TIME,TIME,DAY
0,2020-05-15 00:00:00,4135001,bvBOhCH3iADSZry,0.0,0.0,0.0,6316803.0,4135001,HmiyD2TTLFNqkNe,25.184316,22.857507,0.0,5,2020-05-15 00:00:00,0.00,15
1,2020-05-15 00:15:00,4135001,bvBOhCH3iADSZry,0.0,0.0,0.0,6316803.0,4135001,HmiyD2TTLFNqkNe,25.084589,22.761668,0.0,5,2020-05-15 00:15:00,0.25,15
2,2020-05-15 00:30:00,4135001,bvBOhCH3iADSZry,0.0,0.0,0.0,6316803.0,4135001,HmiyD2TTLFNqkNe,24.935753,22.592306,0.0,5,2020-05-15 00:30:00,0.50,15
3,2020-05-15 00:45:00,4135001,bvBOhCH3iADSZry,0.0,0.0,0.0,6316803.0,4135001,HmiyD2TTLFNqkNe,24.846130,22.360852,0.0,5,2020-05-15 00:45:00,0.75,15
4,2020-05-15 01:00:00,4135001,bvBOhCH3iADSZry,0.0,0.0,0.0,6316803.0,4135001,HmiyD2TTLFNqkNe,24.621525,22.165423,0.0,5,2020-05-15 01:00:00,1.00,15


In [25]:
df_plant1a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3155 entries, 0 to 3154
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   DATE_TIME            3155 non-null   datetime64[ns]
 1   PLANT_ID_x           3155 non-null   int64         
 2   SOURCE_KEY_x         3155 non-null   object        
 3   DC_POWER             3155 non-null   float64       
 4   AC_POWER             3155 non-null   float64       
 5   DAILY_YIELD          3155 non-null   float64       
 6   TOTAL_YIELD          3155 non-null   float64       
 7   PLANT_ID_y           3155 non-null   int64         
 8   SOURCE_KEY_y         3155 non-null   object        
 9   AMBIENT_TEMPERATURE  3155 non-null   float64       
 10  MODULE_TEMPERATURE   3155 non-null   float64       
 11  IRRADIATION          3155 non-null   float64       
 12  MONTH                3155 non-null   int64         
 13  DT_TIME              3155 non-nul

In [26]:
df_plant1a.describe()

,PLANT_ID_x,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,PLANT_ID_y,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,MONTH,TIME,DAY
count,3155.0,3155.000000,3155.000000,3155.000000,3.155000e+03,3155.0,3155.000000,3155.000000,3155.000000,3155.000000,3155.000000,3155.000000
mean,4135001.0,2828.998291,276.778938,3146.858362,6.432819e+06,4135001.0,25.561889,31.181901,0.230245,5.516323,12.005071,15.777179
std,0.0,3698.531647,361.579929,2873.834378,6.578766e+04,0.0,3.351493,12.273902,0.301392,0.499813,6.872654,8.560730
min,4135001.0,0.000000,0.000000,0.000000,6.316803e+06,4135001.0,20.398505,18.140415,0.000000,5.000000,0.000000,1.000000
25%,4135001.0,0.000000,0.000000,0.267857,6.376772e+06,4135001.0,22.739631,21.135510,0.000000,5.000000,6.250000,9.000000
50%,4135001.0,285.000000,27.514286,2896.142857,6.437096e+06,4135001.0,24.682629,24.812990,0.028367,6.000000,12.000000,16.000000
75%,4135001.0,5708.562500,559.431250,5777.169643,6.493334e+06,4135001.0,27.946022,41.449613,0.451674,6.000000,18.000000,23.000000
max,4135001.0,12976.000000,1265.157143,8131.000000,6.539009e+06,4135001.0,35.252486,65.545714,1.221652,6.000000,23.750000,31.000000


In [27]:
df_plant2a.head()

,DATE_TIME,PLANT_ID_x,SOURCE_KEY_x,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,PLANT_ID_y,SOURCE_KEY_y,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,MONTH,DAY,DT_TIME,TIME
0,2020-05-15 00:00:00,4136001,rrq4fwE8jgrTyWY,0.0,0.0,280.214286,120964108.0,4136001,iq8k7ZNt4Mwm3w0,27.004764,25.060789,0.0,5,15,2020-05-15 00:00:00,0.00
1,2020-05-15 00:15:00,4136001,rrq4fwE8jgrTyWY,0.0,0.0,0.000000,120964108.0,4136001,iq8k7ZNt4Mwm3w0,26.880811,24.421869,0.0,5,15,2020-05-15 00:15:00,0.25
2,2020-05-15 00:30:00,4136001,rrq4fwE8jgrTyWY,0.0,0.0,0.000000,120964108.0,4136001,iq8k7ZNt4Mwm3w0,26.682055,24.427290,0.0,5,15,2020-05-15 00:30:00,0.50
3,2020-05-15 00:45:00,4136001,rrq4fwE8jgrTyWY,0.0,0.0,0.000000,120964108.0,4136001,iq8k7ZNt4Mwm3w0,26.500589,24.420678,0.0,5,15,2020-05-15 00:45:00,0.75
4,2020-05-15 01:00:00,4136001,rrq4fwE8jgrTyWY,0.0,0.0,0.000000,120964108.0,4136001,iq8k7ZNt4Mwm3w0,26.596148,25.088210,0.0,5,15,2020-05-15 01:00:00,1.00


In [28]:
df_plant2a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3259 entries, 0 to 3258
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   DATE_TIME            3259 non-null   datetime64[ns]
 1   PLANT_ID_x           3259 non-null   int64         
 2   SOURCE_KEY_x         3259 non-null   object        
 3   DC_POWER             3259 non-null   float64       
 4   AC_POWER             3259 non-null   float64       
 5   DAILY_YIELD          3259 non-null   float64       
 6   TOTAL_YIELD          3259 non-null   float64       
 7   PLANT_ID_y           3259 non-null   int64         
 8   SOURCE_KEY_y         3259 non-null   object        
 9   AMBIENT_TEMPERATURE  3259 non-null   float64       
 10  MODULE_TEMPERATURE   3259 non-null   float64       
 11  IRRADIATION          3259 non-null   float64       
 12  MONTH                3259 non-null   int64         
 13  DAY                  3259 non-nul

In [29]:
df_plant2a.describe()

,PLANT_ID_x,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,PLANT_ID_y,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,MONTH,DAY,TIME
count,3259.0,3259.000000,3259.000000,3259.000000,3.259000e+03,3259.0,3259.000000,3259.000000,3259.000000,3259.000000,3259.000000,3259.000000
mean,4136001.0,209.158443,204.664730,2680.058759,1.201762e+08,4136001.0,28.069400,32.772408,0.232737,5.500153,16.003989,11.866523
std,0.0,329.589960,322.299061,2375.472416,8.393823e+06,0.0,4.061556,11.344034,0.312693,0.500077,8.541578,6.929716
min,4136001.0,0.000000,0.000000,0.000000,0.000000e+00,4136001.0,20.942385,20.265123,0.000000,5.000000,1.000000,0.000000
25%,4136001.0,0.000000,0.000000,241.933333,1.210106e+08,4136001.0,24.602135,23.716881,0.000000,5.000000,9.000000,5.750000
50%,4136001.0,0.000000,0.000000,2368.200000,1.210585e+08,4136001.0,26.981263,27.534606,0.019040,6.000000,16.000000,11.750000
75%,4136001.0,356.540000,349.926667,4128.000000,1.210914e+08,4136001.0,31.056757,40.480653,0.438717,6.000000,23.000000,17.875000
max,4136001.0,1414.120000,1378.766667,9148.000000,1.211314e+08,4136001.0,39.181638,66.635953,1.098766,6.000000,31.000000,23.750000


#### 3.2 Feature Dropping:

Now, we see that we have a lot of features from the original set that we won't use, like the `"Plants ID"`, `"SOURCE_KEY"` or the total yield... We drop them!

Shapes for the Plant 1 DFs:

In [30]:
print(df_plant1a.shape)
print(df_plant1b.shape)
print(df_plant1c.shape)

(3155, 16)
(3154, 16)
(3133, 16)


In [31]:
print(df_plant2a.shape)
print(df_plant2b.shape)
print(df_plant2c.shape)

(3259, 16)
(3259, 16)
(3259, 16)


In [32]:
cols = ['PLANT_ID_x', 'SOURCE_KEY_x', 'SOURCE_KEY_y', 'PLANT_ID_y', 'TOTAL_YIELD', 'DT_TIME']

# Plant 1 A-C:
df_plant1a.drop(cols, axis=1, inplace=True)
df_plant1b.drop(cols, axis=1, inplace=True)
df_plant1c.drop(cols, axis=1, inplace=True)

# Plant 2 A-C:
df_plant2a.drop(cols, axis=1, inplace=True)
df_plant2b.drop(cols, axis=1, inplace=True)
df_plant2c.drop(cols, axis=1, inplace=True)

In [33]:
#Checking the shape fo the DFs, for Plant 1:

print(df_plant1a.shape)
print(df_plant1b.shape)
print(df_plant1c.shape)

(3155, 10)
(3154, 10)
(3133, 10)


In [34]:
# Checking the shape for the DFs, for plant 2:

print(df_plant2a.shape)
print(df_plant2b.shape)
print(df_plant2c.shape)

(3259, 10)
(3259, 10)
(3259, 10)


Finally, we are going to rename some of the features, in order to have shorter names and help in the visualization of the data

In [35]:
# For "AMBIENT TEMPERATURE"

dic1 = {"AMBIENT_TEMPERATURE": "AMB_TEMP"}

# Plant 1 A-C:
df_plant1a.rename(columns=dic1, inplace=True)
df_plant1b.rename(columns=dic1, inplace=True)
df_plant1c.rename(columns=dic1, inplace=True)

# Plant 2 A-C:
df_plant2a.rename(columns=dic1, inplace=True)
df_plant2b.rename(columns=dic1, inplace=True)
df_plant2c.rename(columns=dic1, inplace=True)

In [36]:
# For "MODULE TEMPERATURE"

dic2 = {"MODULE_TEMPERATURE": "MOD_TEMP"}

# Plant 1 A-C:
df_plant1a.rename(columns=dic2, inplace=True)
df_plant1b.rename(columns=dic2, inplace=True)
df_plant1c.rename(columns=dic2, inplace=True)

# Plant 2 A-C:
df_plant2a.rename(columns=dic2, inplace=True)
df_plant2b.rename(columns=dic2, inplace=True)
df_plant2c.rename(columns=dic2, inplace=True)

Lets take a view on each DF...

#### Plant 1:

In [37]:
df_plant1a.describe()

,DC_POWER,AC_POWER,DAILY_YIELD,AMB_TEMP,MOD_TEMP,IRRADIATION,MONTH,TIME,DAY
count,3155.000000,3155.000000,3155.000000,3155.000000,3155.000000,3155.000000,3155.000000,3155.000000,3155.000000
mean,2828.998291,276.778938,3146.858362,25.561889,31.181901,0.230245,5.516323,12.005071,15.777179
std,3698.531647,361.579929,2873.834378,3.351493,12.273902,0.301392,0.499813,6.872654,8.560730
min,0.000000,0.000000,0.000000,20.398505,18.140415,0.000000,5.000000,0.000000,1.000000
25%,0.000000,0.000000,0.267857,22.739631,21.135510,0.000000,5.000000,6.250000,9.000000
50%,285.000000,27.514286,2896.142857,24.682629,24.812990,0.028367,6.000000,12.000000,16.000000
75%,5708.562500,559.431250,5777.169643,27.946022,41.449613,0.451674,6.000000,18.000000,23.000000
max,12976.000000,1265.157143,8131.000000,35.252486,65.545714,1.221652,6.000000,23.750000,31.000000


In [38]:
df_plant1b.describe()

,DC_POWER,AC_POWER,DAILY_YIELD,AMB_TEMP,MOD_TEMP,IRRADIATION,MONTH,DAY,TIME
count,3154.000000,3154.000000,3154.000000,3154.000000,3154.000000,3154.000000,3154.000000,3154.000000,3154.000000
mean,2873.720981,281.124073,3230.658151,25.560769,31.182856,0.230310,5.516487,15.773938,12.003091
std,3761.145773,367.646871,2934.746921,3.351434,12.275731,0.301417,0.499807,8.560151,6.872843
min,0.000000,0.000000,0.000000,20.398505,18.140415,0.000000,5.000000,1.000000,0.000000
25%,0.000000,0.000000,1.129464,22.739499,21.134045,0.000000,5.000000,9.000000,6.250000
50%,302.955357,29.261607,3019.187500,24.681477,24.807481,0.028563,6.000000,16.000000,12.000000
75%,5775.531250,565.209375,5911.000000,27.939607,41.449680,0.451723,6.000000,23.000000,18.000000
max,13335.142860,1300.171429,8268.000000,35.252486,65.545714,1.221652,6.000000,31.000000,23.750000


In [39]:
df_plant1c.describe()

,DC_POWER,AC_POWER,DAILY_YIELD,AMB_TEMP,MOD_TEMP,IRRADIATION,MONTH,DAY,TIME
count,3133.000000,3133.000000,3133.000000,3133.000000,3133.000000,3133.000000,3133.000000,3133.000000,3133.000000
mean,3210.102563,313.900096,3407.255472,25.564299,31.237183,0.231856,5.517715,15.797638,11.948691
std,4114.886897,402.009046,3212.990098,3.360132,12.298276,0.301831,0.499766,8.569409,6.867711
min,0.000000,0.000000,0.000000,20.398505,18.140415,0.000000,5.000000,1.000000,0.000000
25%,0.000000,0.000000,0.000000,22.738440,21.130249,0.000000,5.000000,9.000000,6.000000
50%,446.714286,43.214286,2826.428571,24.678800,24.902235,0.030945,6.000000,16.000000,12.000000
75%,6461.571429,632.985714,6435.000000,27.962351,41.660313,0.454001,6.000000,23.000000,17.750000
max,14299.857140,1394.214286,9013.000000,35.252486,65.545714,1.221652,6.000000,31.000000,23.750000


#### Plant 2:

In [40]:
df_plant2a.describe()

,DC_POWER,AC_POWER,DAILY_YIELD,AMB_TEMP,MOD_TEMP,IRRADIATION,MONTH,DAY,TIME
count,3259.000000,3259.000000,3259.000000,3259.000000,3259.000000,3259.000000,3259.000000,3259.000000,3259.000000
mean,209.158443,204.664730,2680.058759,28.069400,32.772408,0.232737,5.500153,16.003989,11.866523
std,329.589960,322.299061,2375.472416,4.061556,11.344034,0.312693,0.500077,8.541578,6.929716
min,0.000000,0.000000,0.000000,20.942385,20.265123,0.000000,5.000000,1.000000,0.000000
25%,0.000000,0.000000,241.933333,24.602135,23.716881,0.000000,5.000000,9.000000,5.750000
50%,0.000000,0.000000,2368.200000,26.981263,27.534606,0.019040,6.000000,16.000000,11.750000
75%,356.540000,349.926667,4128.000000,31.056757,40.480653,0.438717,6.000000,23.000000,17.875000
max,1414.120000,1378.766667,9148.000000,39.181638,66.635953,1.098766,6.000000,31.000000,23.750000


In [41]:
df_plant2b.describe()

,DC_POWER,AC_POWER,DAILY_YIELD,AMB_TEMP,MOD_TEMP,IRRADIATION,MONTH,DAY,TIME
count,3259.000000,3259.000000,3259.000000,3259.000000,3259.000000,3259.000000,3259.000000,3259.000000,3259.000000
mean,226.161981,221.252177,3751.279985,28.069400,32.772408,0.232737,5.500153,16.003989,11.866523
std,348.778224,340.959752,2578.904346,4.061556,11.344034,0.312693,0.500077,8.541578,6.929716
min,0.000000,0.000000,0.000000,20.942385,20.265123,0.000000,5.000000,1.000000,0.000000
25%,0.000000,0.000000,1789.447619,24.602135,23.716881,0.000000,5.000000,9.000000,5.750000
50%,0.000000,0.000000,3601.000000,26.981263,27.534606,0.019040,6.000000,16.000000,11.750000
75%,403.023333,395.973333,5647.935714,31.056757,40.480653,0.438717,6.000000,23.000000,17.875000
max,1359.626667,1325.833333,9333.000000,39.181638,66.635953,1.098766,6.000000,31.000000,23.750000


In [42]:
df_plant2c.describe()

,DC_POWER,AC_POWER,DAILY_YIELD,AMB_TEMP,MOD_TEMP,IRRADIATION,MONTH,DAY,TIME
count,3259.000000,3259.000000,3259.000000,3259.000000,3259.000000,3259.000000,3259.000000,3259.000000,3259.000000
mean,196.286078,192.090443,2559.308704,28.069400,32.772408,0.232737,5.500153,16.003989,11.866523
std,316.257367,309.319904,2273.856007,4.061556,11.344034,0.312693,0.500077,8.541578,6.929716
min,0.000000,0.000000,0.000000,20.942385,20.265123,0.000000,5.000000,1.000000,0.000000
25%,0.000000,0.000000,258.466667,24.602135,23.716881,0.000000,5.000000,9.000000,5.750000
50%,0.000000,0.000000,2302.000000,26.981263,27.534606,0.019040,6.000000,16.000000,11.750000
75%,327.923333,322.213333,3967.928571,31.056757,40.480653,0.438717,6.000000,23.000000,17.875000
max,1297.740000,1265.666667,9354.000000,39.181638,66.635953,1.098766,6.000000,31.000000,23.750000


#### 3.3 Time Considerations:

Now, we have a data set which, in principle, have measurements performed every 15 minutes during a 2 months period. It means that we should have 96 data points fir every day! From the shape of the DFs we can see that the data is not complete! For such reason we need to fill the gaps.

The data is recorded from the 15th May to the 17th June, for a total of 34 days. It meas that we should have a total of 3264 data points for each sensor.

We start creating an array full of the correct times, and then we will add it the month and the day:

In [43]:
t = np.arange(0, 24, 0.25)
times = []
headers = ['TIME', 'DAY', 'MONTH']


# Add days for May
for day in range(15, 32):
    for i in range(96):
        times.append([t[i], day, 5])

# Add days for June
for day in range(1, 18):
    for i in range(96):
        times.append([t[i], day, 6])
        
times = np.array(times)
df_times = pd.DataFrame(data=times, columns=headers)
print(df_times.shape)
df_times.head()

(3264, 3)


,TIME,DAY,MONTH
0,0.00,15.0,5.0
1,0.25,15.0,5.0
2,0.50,15.0,5.0
3,0.75,15.0,5.0
4,1.00,15.0,5.0


Now we can merge the data frame with the data of the panels and the complete date measurements times

In [44]:
# Before merging...
df_plant1a.shape

(3155, 10)

In [45]:
# Count the number of NaNs before merging

df_plant1a.isna().sum()

DATE_TIME      0
DC_POWER       0
AC_POWER       0
DAILY_YIELD    0
AMB_TEMP       0
MOD_TEMP       0
IRRADIATION    0
MONTH          0
TIME           0
DAY            0
dtype: int64

In [46]:
df_plant1a = df_times.merge(df_plant1a, on=['TIME', 'DAY', 'MONTH'], how='left')

# After merging...
df_plant1a.shape

(3264, 10)

In [47]:
df_plant1a.head(15)

,TIME,DAY,MONTH,DATE_TIME,DC_POWER,AC_POWER,DAILY_YIELD,AMB_TEMP,MOD_TEMP,IRRADIATION
0,0.00,15.0,5.0,2020-05-15 00:00:00,0.0,0.0,0.0,25.184316,22.857507,0.0
1,0.25,15.0,5.0,2020-05-15 00:15:00,0.0,0.0,0.0,25.084589,22.761668,0.0
2,0.50,15.0,5.0,2020-05-15 00:30:00,0.0,0.0,0.0,24.935753,22.592306,0.0
3,0.75,15.0,5.0,2020-05-15 00:45:00,0.0,0.0,0.0,24.846130,22.360852,0.0
4,1.00,15.0,5.0,2020-05-15 01:00:00,0.0,0.0,0.0,24.621525,22.165423,0.0
5,1.25,15.0,5.0,2020-05-15 01:15:00,0.0,0.0,0.0,24.536092,21.968571,0.0
6,1.50,15.0,5.0,2020-05-15 01:30:00,0.0,0.0,0.0,24.638674,22.352926,0.0
7,1.75,15.0,5.0,2020-05-15 01:45:00,0.0,0.0,0.0,24.873022,23.160919,0.0
8,2.00,15.0,5.0,2020-05-15 02:00:00,0.0,0.0,0.0,24.936930,23.026113,0.0
9,2.25,15.0,5.0,2020-05-15 02:15:00,0.0,0.0,0.0,25.012248,23.343229,0.0


In [48]:
df_plant1a.tail(15)

,TIME,DAY,MONTH,DATE_TIME,DC_POWER,AC_POWER,DAILY_YIELD,AMB_TEMP,MOD_TEMP,IRRADIATION
3249,20.25,17.0,6.0,2020-06-17 20:15:00,0.0,0.0,5460.0,22.890356,20.784727,0.0
3250,20.50,17.0,6.0,2020-06-17 20:30:00,0.0,0.0,5460.0,22.891966,20.949461,0.0
3251,20.75,17.0,6.0,2020-06-17 20:45:00,0.0,0.0,5460.0,22.887429,21.121948,0.0
3252,21.00,17.0,6.0,2020-06-17 21:00:00,0.0,0.0,5460.0,22.899882,21.383107,0.0
3253,21.25,17.0,6.0,2020-06-17 21:15:00,0.0,0.0,5460.0,22.892598,21.563179,0.0
3254,21.50,17.0,6.0,2020-06-17 21:30:00,0.0,0.0,5460.0,22.996562,21.869773,0.0
3255,21.75,17.0,6.0,2020-06-17 21:45:00,0.0,0.0,5460.0,23.137091,22.125985,0.0
3256,22.00,17.0,6.0,2020-06-17 22:00:00,0.0,0.0,5460.0,22.563179,21.164713,0.0
3257,22.25,17.0,6.0,2020-06-17 22:15:00,0.0,0.0,5460.0,22.199229,20.515273,0.0
3258,22.50,17.0,6.0,2020-06-17 22:30:00,0.0,0.0,5460.0,22.171737,21.080829,0.0


In [49]:
df_plant1a.isna().sum()

TIME             0
DAY              0
MONTH            0
DATE_TIME      109
DC_POWER       109
AC_POWER       109
DAILY_YIELD    109
AMB_TEMP       109
MOD_TEMP       109
IRRADIATION    109
dtype: int64

After the merging, all new data points are filled with NaNs... Since they are missing points, we can replace them by zeros or by the value of the previous measurement. Looking through the data sets, most of the missing values belong to times where no sun light is observed, for example at midnight. For such reasons we will replace all of them by zeros. To do so, we use the function `fillna()` of `pandas`.

In [50]:
df_plant1a.fillna(0, inplace=True)
df_plant1a.isna().sum()

TIME           0
DAY            0
MONTH          0
DATE_TIME      0
DC_POWER       0
AC_POWER       0
DAILY_YIELD    0
AMB_TEMP       0
MOD_TEMP       0
IRRADIATION    0
dtype: int64

We can now do the same for the other DFs...

In [51]:
# Add Missing times:

# Plant 1 B-C:
df_plant1b = df_times.merge(df_plant1b, on=['TIME', 'DAY', 'MONTH'], how='left')
df_plant1c = df_times.merge(df_plant1c, on=['TIME', 'DAY', 'MONTH'], how='left')

# Plant 2 A-C:
df_plant2a = df_times.merge(df_plant2a, on=['TIME', 'DAY', 'MONTH'], how='left')
df_plant2b = df_times.merge(df_plant2b, on=['TIME', 'DAY', 'MONTH'], how='left')
df_plant2c = df_times.merge(df_plant2c, on=['TIME', 'DAY', 'MONTH'], how='left')

# Replace NaNs with 0:

# Plant 1 B-C:
df_plant1b.fillna(0, inplace=True)
df_plant1c.fillna(0, inplace=True)

# Plant 2 A-C:
df_plant2a.fillna(0, inplace=True)
df_plant2b.fillna(0, inplace=True)
df_plant2c.fillna(0, inplace=True)

In [52]:
#Checking the shape fo the DFs, for Plant 1:

print(df_plant1a.shape)
print(df_plant1b.shape)
print(df_plant1c.shape)

# Checking the shape for the DFs, for plant 2:

print(df_plant2a.shape)
print(df_plant2b.shape)
print(df_plant2c.shape)

(3264, 10)
(3264, 10)
(3264, 10)
(3264, 10)
(3264, 10)
(3264, 10)


Now we a have a complete set of features, ready to be modelled!